In [8]:
import tensorflow as tf
import numpy as np

In [18]:
def conv_unit(input_data,w,b,strides):
    conv = tf.nn.conv2d(input_data, w,strides=[1, strides, strides, 1], padding = 'SAME')
    conv_bias = tf.nn.bias_add(conv,b)
    conv_relu = tf.nn.relu(conv_bias)
    
    conv_max = tf.nn.max_pool(conv_relu, ksize=[1,3,3,1], strides=[1, 2, 2, 1], padding = 'SAME')
    return conv_max

In [26]:
def inception_unit(input_data, weights):
    inception_in = input_data
    # conv 1x1 + s1
    inception_1x1 = tf.nn.conv2d(inception_in, weights['inception_1x1'], strides=[1, 1, 1, 1], padding='SAME')
    inception_1x1 = tf.nn.bias_add(inception_1x1, weights['inception_1x1_b'])
    inception_1x1 = tf.nn.relu(inception_1x1)
    
    # conv 3x3 + s1
    inception_3x3_reduce = tf.nn.conv2d(inception_in,weights['inception_3x3_reduce'],strides=[1, 1, 1, 1], padding='SAME')
    inception_3x3_reduce = tf.nn.bias_add(inception_3x3_reduce, weights['inception_3x3_reduce_b'])
    inception_3x3_reduce = tf.nn.relu(inception_3x3_reduce)
    inception_3x3 = tf.nn.conv2d(inception_3x3_reduce, weights['inception_3x3'],strides=[1, 1, 1, 1],padding='SAME')
    inception_3x3 = tf.nn.bias_add(inception_3x3, weights['inception_3x3_b'])
    inception_3x3 = tf.nn.relu(inception_3x3)
    
    # conv 5x5 + s1
    inception_5x5_reduce = tf.nn.conv2d(inception_in,weights['inception_5x5_reduce'],strides=[1, 1, 1, 1], padding='SAME')
    inception_5x5_reduce = tf.nn.bias_add(inception_5x5_reduce, weights['inception_5x5_reduce_b'])
    inception_5x5_reduce = tf.nn.relu(inception_5x5_reduce)
    inception_5x5 = tf.nn.conv2d(inception_5x5_reduce, weights['inception_5x5'],strides=[1, 1, 1, 1],padding='SAME')
    inception_5x5 = tf.nn.bias_add(inception_5x5, weights['inception_5x5_b'])
    inception_5x5 = tf.nn.relu(inception_5x5)
    
    # Maxpool 
    inception_MaxPool = tf.nn.max_pool(inception_in, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')
    inception_MaxPool = tf.nn.conv2d(inception_MaxPool, weights['inception_MaxPool'], strides=[1, 1, 1, 1], padding='SAME')
    inception_MaxPool = tf.nn.bias_add(inception_MaxPool, weights['inception_MaxPool_b'])
    inception_MaxPool = tf.nn.relu(inception_MaxPool)
    
    inception_out = tf.concat(axis = 3, values = [inception_1x1, inception_3x3, inception_5x5, inception_MaxPool])
    
    return inception_out

In [27]:
X = tf.placeholder(tf.float32,[None,224, 224, 3])
y = tf.placeholder(tf.float32,[None,8])

other_conv = {
    'conv_7x7':tf.Variable(tf.truncated_normal([7, 7, 3, 64],stddev=0.001)),
    'conv_7x7_b':tf.Variable(tf.truncated_normal([64],stddev=0.001)),
    'conv_3x3':tf.Variable(tf.truncated_normal([3, 3, 64, 192],stddev=0.001)),
    'conv_3x3_b':tf.Variable(tf.truncated_normal([192],stddev=0.001))
}
inception_3a = {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 192, 64],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([64],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 192, 64],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([64],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 64, 128],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([128],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 192, 16],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([16],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 16, 32],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([32],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,192,32],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([32],stddev=0.001))
}
inception_3b = {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 256, 128],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([128],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 256, 128],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([128],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 128, 192],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([192],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 256, 32],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([32],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 32, 96],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([96],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,256,64],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([64],stddev=0.001))
}
inception_4a = {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 480, 192],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([192],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 480, 96],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([96],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 96, 208],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([208],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 480, 16],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([16],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 16, 48],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([48],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,480,64],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([64],stddev=0.001))
}
inception_4b= {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 512, 160],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([160],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 512, 112],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([112],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 112, 224],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([224],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 512, 24],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([24],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 24, 64],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([64],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,512,64],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([64],stddev=0.001))
}
inception_4c = {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 512, 128],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([128],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 512, 128],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([128],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 128, 256],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([256],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 512, 24],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([24],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 24, 64],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([64],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,512,64],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([64],stddev=0.001))
}
inception_4d = {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 512, 112],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([112],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 512, 144],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([144],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 144, 288],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([288],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 512, 32],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([32],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 32, 64],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([64],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,512,64],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([64],stddev=0.001))
}
inception_4e = {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 528, 256],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([256],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 528, 160],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([160],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 160, 320],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([320],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 528, 32],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([32],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 32, 128],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([128],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,528,128],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([128],stddev=0.001))
}
inception_5a = {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 832, 256],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([256],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 832, 160],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([160],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 160, 320],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([320],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 832, 32],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([32],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 32, 128],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([128],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,832,128],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([128],stddev=0.001))
}
inception_5b = {
    'inception_1x1':tf.Variable(tf.truncated_normal([1, 1, 832, 384],stddev=0.001)),
    'inception_1x1_b':tf.Variable(tf.truncated_normal([384],stddev=0.001)),
    'inception_3x3_reduce':tf.Variable(tf.truncated_normal([1, 1, 832, 192],stddev=0.001)),
    'inception_3x3_reduce_b':tf.Variable(tf.truncated_normal([192],stddev=0.001)),
    'inception_3x3':tf.Variable(tf.truncated_normal([3, 3, 192, 384],stddev=0.001)),
    'inception_3x3_b':tf.Variable(tf.truncated_normal([384],stddev=0.001)),
    'inception_5x5_reduce':tf.Variable(tf.truncated_normal([1, 1, 832, 48],stddev=0.001)),
    'inception_5x5_reduce_b':tf.Variable(tf.truncated_normal([48],stddev=0.001)),
    'inception_5x5':tf.Variable(tf.truncated_normal([3, 3, 48, 128],stddev=0.001)),
    'inception_5x5_b':tf.Variable(tf.truncated_normal([128],stddev=0.001)),
    'inception_MaxPool':tf.Variable(tf.truncated_normal([1,1,832,128],stddev=0.001)),
    'inception_MaxPool_b':tf.Variable(tf.truncated_normal([128],stddev=0.001))
}

In [29]:
# 第一层
conv7_7 = conv_unit(X,other_conv['conv_7x7'],other_conv['conv_7x7_b'],2)

# 第二层
conv3_3 = conv_unit(conv7_7,other_conv['conv_3x3'],other_conv['conv_3x3_b'],1)

# inception 3a
conv_3a = inception_unit(conv3_3,inception_3a)

#inception 3b
conv_3b = inception_unit(conv_3a,inception_3b)

conv_max = tf.nn.max_pool(conv_3b,ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

conv_4a = inception_unit(conv_max,inception_4a)

conv_4b = inception_unit(conv_4a,inception_4b)

conv_4c = inception_unit(conv_4b,inception_4c)

conv_4d = inception_unit(conv_4c,inception_4d)

conv_4e = inception_unit(conv_4d,inception_4e)

conv_max = tf.nn.max_pool(conv_4e,ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

conv_5a = inception_unit(conv_max,inception_5a)

conv_5b = inception_unit(conv_5a,inception_5b)

avg_pool = tf.reduce_mean(conv_5b, [1, 2], keep_dims=True)



Instructions for updating:
keep_dims is deprecated, use keepdims instead
